# Real antibody data example
In this notebook we fit real data from a monoclonal antibody deep mutational scanning experiment.
The monoclonal antibody in question is REGN10933, and the deep mutational scanning is against the SARS-CoV-2 Delta spike, using a library designed to contain only mutations that are thought to be well tolerated in a lentiviral platform.

First, import the Python modules:

In [1]:
import time

import altair as alt

import polyclonal

import pandas as pd

## Get the data to fit
Now we read the deep mutational scanning measurements, which quantify the "probability of escape" (fraction not neutralized) for each variant.
Then let's look at some key columns:

In [2]:
# read data w `na_filter=None` so empty aa_substitutions read as such rather than NA
data_to_fit = pd.read_csv("LibB_2022-03-02_thaw-3_REGN10933_1.csv", na_filter=None)

data_to_fit[
    [
        "antibody_concentration",
        "barcode",
        "aa_substitutions",
        "n_aa_substitutions",
        "prob_escape",
        "prob_escape_uncensored",
        "no-antibody_count",
    ]
]

,antibody_concentration,barcode,aa_substitutions,n_aa_substitutions,prob_escape,prob_escape_uncensored,no-antibody_count
0,0.15,GAAGATCACTGGTCGA,P26T S477D,2,0.7746,0.7746,12589
1,0.15,TGGCTGCAACGTTCAA,K417L T859N,2,0.8118,0.8118,7087
2,0.15,ATGTACATTACTGACA,S151N V483L,2,0.3299,0.3299,9121
3,0.15,GTGGAGGAATCTCCCT,,0,0.2986,0.2986,10001
4,0.15,ATCAGCTCGTTCTAAA,Q218R K417R Q677E G842V,4,0.7873,0.7873,3787
...,...,...,...,...,...,...,...
59347,5.58,TTTTGACAAGCGATTT,G1167V,1,0.0000,0.0000,134
59348,5.58,TTTTTGAATTTATGTC,R646G,1,0.0000,0.0000,286
59349,5.58,TTTTTGTTATAGCCCC,V615M A1020T F1103L L1200F,4,0.0000,0.0000,448
59350,5.58,TTTTTTAATTAACGCA,F1109Y,1,0.0000,0.0000,203


Each row in the above data frame is a different variant (defined by it's barcode) that has some set of amino-acid substitutions and has it's probability of escape (fraction not neutralized) measured at a given antibody concentration (which can be in arbitrary units).

The probability of escape should in reality be between 0 and 1 (since you can't have more than complete escape), but due to noise in the measurements the values will sometimes be >1.
We therefore have the censored probabilities of escape (`prob_escape`) that we will actually fit, plus the uncensored values.
We expect some uncensored values to be >1, but it's good to confirm that on average they aren't much different than the censored values (if they are, could indicate some experimental problem).

Note also that the experimental estimation of the probability of escape is ultimately based on sequencing counts.
If the no-antibody count is not sufficiently high, the measurements will be very noisy.
The above data have been pre-filtered to only include variants with at least moderately high pre-antibody counts.

Typically we have the same number of variants at each antibody concentration, and they should have the same statistics for the no-antibody counts as a single no-antibody control is used alongside all the antibody selections.
The below table confirms some key points are as expected:
 - we have the same variants for each concentration (although this is not a strict requirement for model fitting)
 - the mean probability of escape (fraction not neutralized) decreases with increasing antibody concentration
 - the uncensored probabilities of escape are _on average_ very similar to the censored ones we will actually fit (if this isn't true it's a red flag about your data!)
 - the mean and minimum no-antibody counts are the same for all concentrations (as they all use the same no-antibody control) and are reasonably large, outside the range where statistical noise is expected to have a major effect.

In [3]:
display(
    data_to_fit.groupby("antibody_concentration")
    .aggregate(
        n_variants=pd.NamedAgg("barcode", "nunique"),
        mean_prob_escape=pd.NamedAgg("prob_escape", "mean"),
        mean_prob_escape_uncensored=pd.NamedAgg("prob_escape_uncensored", "mean"),
        mean_no_antibody_count=pd.NamedAgg("no-antibody_count", "mean"),
        min_no_antibody_count=pd.NamedAgg("no-antibody_count", "min"),
    )
    .round(2)
)

,n_variants,mean_prob_escape,mean_prob_escape_uncensored,mean_no_antibody_count,min_no_antibody_count
antibody_concentration,,,,,
0.15,19784,0.29,0.30,502.29,106
1.39,19784,0.05,0.05,502.29,106
5.58,19784,0.01,0.01,502.29,106


We will also plot the mean probability of escape across variants with each numbers of mutations.
The expectation is that variants with more mutations will tend to have more escape, and again the censored and uncensored values should look similar.
You can see that is true in plot below.
Note that the y-axis is a symlog scale; mouse over points for details:

In [4]:
# NBVAL_IGNORE_OUTPUT

max_aa_subs = 4  # group if >= this many substitutions

mean_prob_escape = (
    data_to_fit.assign(
        n_subs=lambda x: (
            x["n_aa_substitutions"]
            .clip(upper=max_aa_subs)
            .map(lambda n: str(n) if n < max_aa_subs else f">{max_aa_subs - 1}")
        )
    )
    .groupby(["antibody_concentration", "n_subs"], as_index=False)
    .aggregate({"prob_escape": "mean", "prob_escape_uncensored": "mean"})
    .rename(
        columns={
            "prob_escape": "censored to [0, 1]",
            "prob_escape_uncensored": "not censored",
        }
    )
    .melt(
        id_vars=["antibody_concentration", "n_subs"],
        var_name="censored",
        value_name="probability escape",
    )
)

mean_prob_escape_chart = (
    alt.Chart(mean_prob_escape)
    .encode(
        x=alt.X("antibody_concentration"),
        y=alt.Y(
            "probability escape",
            scale=alt.Scale(type="symlog", constant=0.05),
        ),
        column=alt.Column("censored", title=None),
        color=alt.Color("n_subs", title="n substitutions"),
        tooltip=[
            alt.Tooltip(c, format=".3g") if mean_prob_escape[c].dtype == float else c
            for c in mean_prob_escape.columns
        ],
    )
    .mark_line(point=True, size=0.5)
    .properties(width=200, height=125)
    .configure_axis(grid=False)
)

mean_prob_escape_chart

alt.Chart(...)

## Fit `polyclonal` model
We will now fit a `polyclonal` model to the data, and then also bootstrap the fit to get some error estimates.

In general, an important consideration when fitting these models is how many epitopes to include.
However, here it's just a **monoclonal** antibody (not polyclonal serum), so we know we should just have one epitope.

First, fit a model to all the data.
We use an alphabet that includes stop (`*`) and gap (`-`) characters in addition to the 20 amino acids, in case some variants have such characters:

In [5]:
# NBVAL_IGNORE_OUTPUT

model = polyclonal.Polyclonal(
    # `polyclonal` expects the concentration column to be named "concentration"
    data_to_fit=data_to_fit.rename(columns={"antibody_concentration": "concentration"}),
    n_epitopes=1,
    alphabet=polyclonal.AAS_WITHSTOP_WITHGAP,
)

_ = model.fit(logfreq=100)

# First fitting site-level model.
# Starting optimization of 1154 parameters at Sun Apr 10 11:03:52 2022.
       step   time_sec       loss   fit_loss reg_escape  regspread
          0   0.032539     9128.1     9128.1          0          0
        100     3.6739     933.07     930.65     2.4156          0
        200     7.4154     931.57     929.02     2.5417          0
        221     8.2107     931.54     928.98     2.5569          0
# Successfully finished at Sun Apr 10 11:04:00 2022.
# Starting optimization of 5875 parameters at Sun Apr 10 11:04:00 2022.
       step   time_sec       loss   fit_loss reg_escape  regspread
          0    0.03518     1435.2       1426     9.1836 1.4644e-31
        100     3.9202       1153       1104     17.255     31.779
        200     7.7631     1151.1     1100.4     17.487     33.167
        240     9.2112     1151.1     1100.6     17.461     33.015
# Successfully finished at Sun Apr 10 11:04:09 2022.


Now we fit a bootstrapped model, using as the "root" model the one we just fit to all the data.
This enables estimates on parameter error:

In [6]:
# NBVAL_IGNORE_OUTPUT

# for publication data, you may want more like 50-100 bootstrap samples,
# but here we just fit 20 to make things faster.
n_bootstrap_samples = 20

bootstrap_model = polyclonal.PolyclonalCollection(
    root_polyclonal=model,
    n_bootstrap_samples=n_bootstrap_samples,
    n_threads=2,  # specify number as here, otherwise uses all available
)

print(f"Starting fitting at {time.asctime()}")
n_fit, n_failed = bootstrap_model.fit_models()
assert n_fit == n_bootstrap_samples and n_failed == 0
print(f"Finished fitting at {time.asctime()}")

Starting fitting at Sun Apr 10 11:04:20 2022
Finished fitting at Sun Apr 10 11:06:01 2022


## Visualize how mutations affect escape
Now we look at the results of the fitting.

First, we just look at the activity of the epitope.
For a monoclonal antibody with just one epitope, this isn't very complicated to interpret: the one epitope should have significantly positive activity.
If it doesn't, something is wrong.
Here, the activity of the epitope is positive as expected:

In [7]:
bootstrap_model.activity_wt_df.round(2)

,epitope,activity_mean,activity_std
0,1,2.97,0.01


The more interesting measurements are how mutations affect escape.

Importantly, in visualizing these results, there is an important parameter we need to consider: the number of times that mutation is seen in a variant.
A mutation that is seen in just one variant is more susceptible to being poorly measured as there are less data supporting it.
Also, the bootstrapping may not capture "noise" for such mutations very well since there aren't multiple measurements to bootstrap.
So we set a `min_times_seen` parameter that tells us how many variants must have the mutation in order to show results for it:

In [8]:
min_times_seen = 3

Now we create a heat map showing how each mutation affects escape:

In [9]:
# NBVAL_IGNORE_OUTPUT

bootstrap_model.mut_escape_heatmap(init_min_times_seen=min_times_seen)

alt.VConcatChart(...)

In the heatmap above, each cell is a different mutation (those without measurements are grayed out).
The `x` characters mark the wildtype identity, and you can mouse over points for details.
You can use the zoom bar to zoom to specific regions.
You can also see that the `min_times_seen` parameter is initially set to the value we determined above, but you can use the slider to adjust the value to see mutations seen in fewer variants (or in more variants, if you increase value).
Most sites don't have much escape, so you can use the `percent_max_cutoff` slider to just get sites where there is a mutation with lots of escape (specifically, this slider only shows sites with mutations that have escape $\ge$ some percent of the maximal escaping mutation).

Using that slider, we can see that the sites of strongest escape are 417, 453, 455, 476, 477, 484, 486, 487, 489, and 493.
These correspond closely to the sites of escape found by yeast-display deep mutational scanning of the REGN10933 antibody by [Starr et al (2021)](https://www.science.org/doi/10.1126/science.abf9302) (see Fig 1A of that paper) and [Baum et al (2020)](https://www.science.org/doi/full/10.1126/science.abd0831) (see Table 2 of that paper).
Specifically, those are the sites where escape mutations come up if the slider in the above image is adjusted to `percent_max_cutoff` of 70%.
The top sites can also be accessed programmatically from the bootstrapped polyclonal model:

In [10]:
display(
    bootstrap_model.mut_escape_df.query("times_seen >= @min_times_seen")
    .sort_values("escape_mean", ascending=False)
    .groupby("site", as_index=False, sort=False)
    .first(1)[["site", "escape_mean"]]
    .rename(columns={"escape_mean": "largest_mutation_escape_at_site"})
    .head(n=10)
    .round(2)
)

,site,largest_mutation_escape_at_site
0,486,4.42
1,417,4.08
2,489,4.07
3,487,4.03
4,476,3.93
5,493,3.62
6,484,3.57
7,455,3.56
8,453,3.41
9,477,3.39


The heatmap above is useful, but is a very dense display of information.
We can also display the site-level summaries of escape.
Again, the choice of `min_times_seen` is crucial, as it determines which mutations are used to get the site-level statistics:

In [11]:
# NBVAL_IGNORE_OUTPUT

bootstrap_model.mut_escape_lineplot(
    mut_escape_site_summary_df_kwargs={"min_times_seen": min_times_seen},
)

alt.VConcatChart(...)

The above plot initially shows the **total** positive escape at each site, which is influenced both by how strongly mutations at a site escape and by how many amino acids mutations at that site are in the library (usually sites more tolerant of mutations will have more amino acids represented).
You can adjust the metric that is show to for instance be the mean or max escape at a site, and also again use the `percent_max_cutoff` slider to zoom in on the biggest escape sites.
Doing this, you will again see that the same known escape sites typically come out in the top set.